### Domino : parties avec MC

In [2]:
import numpy as np
import random
from random import randint

def coup_un(A,i,j):
    if A[i,j] == 0 and A[i+1,j] == 0:
        A[i,j] = 1
        A[i+1,j] = 1

def coup_deux(A,i,j):
    if A[i,j] == 0 and A[i,j+1] == 0:
        A[i,j] = 2
        A[i,j+1] = 2

def coups_possibles_un(A):
    liste_possible = []
    for i in range(0,len(A)-1):
        for j in range(0,len(A)):
            if A[i,j] ==0 and A[i+1,j] == 0:
                liste_possible.append((i,j))
    return liste_possible

def coups_possibles_deux(A):
    liste_possible = []
    for i in range(0,len(A)):
        for j in range(0,len(A)-1):
            if A[i,j] ==0 and A[i,j+1] == 0:
                liste_possible.append((i,j))
    return liste_possible

def rand_domineering(A, player):
    while 1:
        if player == 1:
            if coups_possibles_un(A) != []:
                play = random.choice(coups_possibles_un(A))
                coup_un(A, play[0], play[1])
                player = 2
            else:
                return player
        if player == 2:
            if coups_possibles_deux(A) != []:
                play = random.choice(coups_possibles_deux(A))
                coup_deux(A, play[0], play[1])
                player = 1
            else:
                return player

def mc_domineering(A, player):
    while 1:
        if player == 1:
            if coups_possibles_un(A) != []:
                play = montecarlo(A, player)
                coup_un(A, play[0], play[1])
                player = 2
            else:
                return player
        if player == 2:
            if coups_possibles_deux(A) != []:
                play = montecarlo(A, player)
                coup_deux(A, play[0], play[1])
                player = 1
            else:
                return player

def montecarlo(A, player):
    B = A[:]
    if player == 1:
        moves = coups_possibles_un(A)
        if moves != []:
            for coup in moves:
                victoire = np.zeros(len(moves))
                for i in range(0,20):
                    coup_un(A, coup[0], coup[1])
                    loser = rand_domineering(A, 2)
                    if loser == 2:
                        victoire[moves.index(coup)] = victoire[moves.index(coup)] + 1
                    A = B[:]
    if player == 2:
        moves = coups_possibles_deux(A)
        if moves != []:
            for coup in moves:
                victoire = np.zeros(len(moves))
                for i in range(0,20):
                    coup_deux(A, coup[0], coup[1])
                    loser = rand_domineering(A, 1)
                    if loser == 1:
                        victoire[moves.index(coup)] = victoire[moves.index(coup)] + 1
                    A = B[:]
    victoire = victoire / 20
    max = 0
    for i in range(0, len(victoire)):
        if victoire[i] > max:
            max = victoire[i]
            coupchoisi = moves[i]
    return coupchoisi

def rand_vs_mc_domineering(A, player):
    while 1:
        if player == 1:
            if coups_possibles_un(A) != []:
                play = montecarlo(A, player)
                coup_un(A, play[0], play[1])
                player = 2
            else:
                return player
        if player == 2:
            if coups_possibles_deux(A) != []:
                play = random.choice(coups_possibles_deux(A))
                coup_deux(A, play[0], play[1])
                player = 1
            else:
                return player

losers_ran = []
losers_mc = []
losers_mc_vs_mc = []

for i in range(0,100):
    player = randint(1,2)
    damier = np.zeros((8, 8))
    losers_ran.append(rand_domineering(damier, player))

for i in range(0,100):
    player = randint(1,2)
    damier = np.zeros((8, 8))
    losers_mc.append(mc_domineering(damier, player))

for i in range(0, 100):
    player = randint(1, 2)
    damier = np.zeros((8, 8))
    losers_mc_vs_mc.append(rand_vs_mc_domineering(damier, player))

print("In Random Domineering, Player 1 loses (%) : ", losers_ran.count(1)/100)
print("In Monte Carlo Domineering, Player 1 loses (%) : ", losers_mc.count(1)/100)
print("In Monte Carlo (Player 1) vas Random (Player 2) Domineering, Player 1 loses : (%) ", losers_mc_vs_mc.count(1)/100)

In Random Domineering, Player 1 loses (%) :  0.49
In Monte Carlo Domineering, Player 1 loses (%) :  0.47
In Monte Carlo (Player 1) vas Random (Player 2) Domineering, Player 1 loses : (%)  0.0


### Model DL 1

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D

def get_model(board_size):
    model = Sequential()
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(board_size, board_size, 3), padding="same"))
    model.add(Conv2D(64, (3, 3), activation='relu', padding="same"))
    model.add(Conv2D(64, (3, 3), activation='relu', padding="same"))
    model.add(Conv2D(64, (3, 3), activation='relu', padding="same"))
    model.add(Conv2D(64, (3, 3), activation='relu', padding="same"))
    model.add(Conv2D(1, (3, 3), activation='relu', padding="same"))
    model.add(Flatten())
    model.add(Dense(board_size**2, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])
    return model

def prepare_data(row, board_size):
    board = row[:board_size**2]
    flipped_board = row[board_size**2:2*board_size**2]
    turn = row[2*board_size**2:3*board_size**2]
    output = row[3*board_size**2:]

    board = np.array(board).reshape((board_size, board_size)).T
    flipped_board = np.array(flipped_board).reshape((board_size, board_size)).T
    turn = np.array(turn).reshape((board_size, board_size)).T
    output = np.array(output)

    state = np.zeros((board_size, board_size, 3))
    state[:,:,0] = board
    state[:,:,1] = flipped_board
    state[:,:,2] = turn

    return state, output

def prepare_data_lr(row, board_size):
    data = prepare_data(row, board_size)
    state = data[0]
    state = np.fliplr(state)

    output = data[1]
    output = np.array(output).reshape((board_size, board_size)).T
    output = np.fliplr(output)

    player = int(state[:, :, 2][0][0])
    if player == 0:  # vertical
        output = output.T
        output = output.ravel()
        return state, output
    if player == 1:  # horizental
        permutation = [7, 0, 1, 2, 3, 4, 5, 6]
        i = np.argsort(permutation)
        output_moved = output[:, i]
        output_moved = output_moved.T
        output_moved = output_moved.ravel()
        return state, output_moved

def prepare_data_ud(row, board_size):
    data = prepare_data(row, board_size)
    state = data[0]
    state = np.flipud(state)

    output = data[1]
    output = np.array(output).reshape((board_size, board_size)).T
    output = np.flipud(output)

    player = int(state[:, :, 2][0][0])
    if player == 1:  # horizental
        output = output.T
        output = output.ravel()
        return state, output
    if player == 0:  # vertical
        output_moved = np.roll(output, 7, axis=0)
        output_moved = output_moved.T
        output_moved = output_moved.ravel()
        return state, output_moved

filename = 'domineering.csv'
df = pd.read_csv(filename, sep=',', na_values=[""], header=None)
board_size = 8

data = df.values.tolist()
train = data[:int(0.7*len(data))]
test = data[int(0.7*len(data)):]

X = []

for t in train:
    row = prepare_data(t, board_size)
    X.append(row)
    row_lr = prepare_data_lr(t, board_size)
    X.append(row_lr)
    row_ud = prepare_data_ud(t, board_size)
    X.append(row_ud)

X_train = np.array([k[0] for k in X])
Y_train = np.array([k[1] for k in X])

XT = []

for t in test:
    row = prepare_data(t, board_size)
    XT.append(row)
    row_lr = prepare_data_lr(t, board_size)
    XT.append(row_lr)
    row_ud = prepare_data_ud(t, board_size)
    X.append(row_ud)

X_test = np.array([k[0] for k in XT])
Y_test = np.array([k[1] for k in XT])

model = get_model(board_size)

model.fit(X_train, Y_train, validation_split=0.1, epochs=20)
pred_test = model.predict(X_test)
acc = accuracy_score(np.argmax(Y_test, axis=1), np.argmax(pred_test, axis=1))
print("Accuracy : %s " %acc)

Train on 53211 samples, validate on 5913 samples
Epoch 1/20
53211/53211 [==============================] - 65s 1ms/step - loss: 4.1446 - acc: 0.0205 - val_loss: 4.1392 - val_acc: 0.0211
Epoch 2/20
53211/53211 [==============================] - 78s 1ms/step - loss: 4.1375 - acc: 0.0217 - val_loss: 4.1368 - val_acc: 0.0205
Epoch 3/20
37728/53211 [====================>.........] - ETA: 21s - loss: 3.9395 - acc: 0.0546

KeyboardInterrupt: 